# Capture Traces for 21.12

In [1]:
import os

In [2]:
## repository of saves
save_rep = "E:/DP_database/database"
os.makedirs(f'{save_rep}/figures/get_weights/', exist_ok=True)
os.makedirs(f'{save_rep}/figures/SPA/', exist_ok=True)
os.makedirs(f'{save_rep}/figures/get_weights/methods/', exist_ok=True)

## Libraries and Other Notebooks

In [3]:
import matplotlib.pyplot as plt

In [4]:
%run "./File_Uploader.ipynb"

C:\Users\Samuel\AppData\Local\Temp\ipykernel_3972\4114886631.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


✔️ The File uploader succesfuly imported.


In [5]:
%run "./ANN_Observer.ipynb"

✔️ The Setuper class succesfuly imported.
✔️ The ANN_Observer class succesfuly imported.


In [6]:
%run "./Analyser.ipynb"    

✔️ The Analyser succesfuly runned.


## Function Definitions

### Scope Setup

In [7]:
scope : cw.scope = None
target: cw.target = None
cw_setuper: CW_Setuper
def setup(makedir):
    global scope
    global target
    global cw_setuper
    make_dir = makedir
    re_init_config = {"MAKEDIR" : f'./{make_dir}/'}
    cw_setuper = CW_Setuper(firmware_reinit = True, re_init_dict  = re_init_config)
    scope = cw_setuper.scope
    target = cw_setuper.target
    return scope, target, cw_setuper

### Measurement

In [8]:
def capture_target(scope, epochs = 3000, decimate = 1, reverse = False, runName="_722_BasicMLP",trace_len_mult:int=1):
    cmd = 'p'
    if reverse:
        cmd = 'o'
    name = f"{runName}_Dec{decimate}-ntg{decimate}-{epochs}" #ntg no-target-reset
     
    scope_setup(samples=24429*trace_len_mult, decimate=decimate)    
    traces, rand_inputs = measure_traces(cmd=cmd, epochs=epochs)
    waves = []
    out_data = []
    for i in traces:
        waves.append(i[0])
        out_data.append(i[2])
    global save_rep
    files = save_data(waves, rand_inputs, out_data[0], path=save_rep, name_extended = name)
    #upload_file_to_gdrive(files,  dir_path = "E:/DP_database/database/")
    #Analysis
    print(out_data[0])
    ith_weight = 0
    start = ith_weight*epochs

    diff_avg, avg_wave, var, ____ = createDiffWave(waves, runName)
    return rand_inputs, waves, out_data[0], diff_avg, avg_wave, var

### Correlation calculation

In [13]:
def calc_corr(waves, rand_inputs, _func, _name, epochs, trace_len):
    ncorr_all, p_values_matrix = _func(rand_inputs, np.array(waves)[:,:], n_traces = epochs, trace_len = trace_len, ith_weight = 0, calc_p_value=False)
    global save_rep
    np.save(f"{save_rep}/corrs_{_name}.npy", np.array(ncorr_all))
    plot_mult(np.array(ncorr_all)[:,:], _type = f"Corrs_{_name}", ylim=[-1,1], graph_title=f"Corrs_{_name}", ylabel="Power")
    return ncorr_all, p_values_matrix

In [14]:
def capture_correlate(scope, epochs_list = [3000], decimate = 1, reverse = False, runName="_722_BasicMLP", trace_len_mult:int=1):
    for i in trange(len(epochs_list), desc='Capturing and Analysing the traces:'):
        epochs = epochs_list[i]
        rand_inputs, waves, out_data, diff_avg, avg_wave, var = capture_target(scope=scope, epochs=epochs, decimate=decimate,
                                                          reverse=reverse, runName=runName,trace_len_mult=trace_len_mult)
        corr_HW8, HW8_pValues = calc_corr(waves=waves, rand_inputs= np.array(rand_inputs), _func=HW_8bit_ANN_CPA, _name="_HW8", epochs=epochs, trace_len=24000)
        corr_HW32, HW32_pValues = calc_corr(waves=waves, rand_inputs= np.array(rand_inputs), _func=HW_32bit_ANN_CPA, _name="_HW32", epochs=epochs, trace_len=24000)
        corr_Abs8, Abs8_pValues = calc_corr(waves=waves, rand_inputs= np.array(rand_inputs), _func=Abs_8bit_ANN_CPA, _name="_Abs8", epochs=epochs, trace_len=24000)
        corr_Abs32, Abs32_pValues = calc_corr(waves=waves, rand_inputs= np.array(rand_inputs), _func=Abs_32bit_ANN_CPA, _name="_Abs32", epochs=epochs, trace_len=24000)
    global cw_setuper
    global target
    cw_setuper._scope_disc_()
    scope = None
    target = None
    return corr_HW8, corr_HW32, corr_Abs8, corr_Abs32, rand_inputs, waves, out_data, diff_avg, avg_wave, var

### Extraction of secret values

In [11]:
def analyse_corr(corr_map: np.ndarray, weights: list[int], stop_num = 5, maxims_map = None, _index = 0, peak_range = 0.3,
                 show_info=True, mountain_half_dist = 5, level = 3, saveName = None):
    index = _index
    global save_rep
    for secret in weights:
        
        print(f"Hello {secret}")
        secret_value, time_sample, correlation, localMax_vector, colmn_maxVector, nextIndex, maxims_map = get_weight(correlation_matrix=corr_map,
                                                                                                                     peak_range=peak_range,
                                                                                                                     show_info=True,
                                                                                                                     lastIndex=index,
                                                                                                                     map_2dgrMaxs=maxims_map,
                                                                                                                     mountain_half_dist=mountain_half_dist,
                                                                                                                     level=level)
    
        index = nextIndex
        figure, axis = plt.subplots(1, 2)
        # Comparison to real correlation
        axis[0].plot(colmn_maxVector, color='b', label='agregate_of_maxs_corrs', linewidth=0.5)
        axis[0].plot(corr_map[secret], color='g', label=f'secret_val_corr = {secret}', linewidth=0.5)
        #axis[0].legend(loc='upper right')
        axis[0].legend(bbox_to_anchor=(0.75, 1.15), ncol=2)
        axis[0].set_title("All - Real")
        plt.xlabel("Time Sample")
        plt.ylabel("Correlation")
        plt.axvline(x = index, color = 'orange')

        # Comparison to found correlation
        axis[1].plot(colmn_maxVector, color='b', label='agregate_of_maxs_corrs', linewidth=0.5)
        axis[1].plot(corr_map[secret_value], color='r', label=f'found_corr = {secret_value}', linewidth=0.5)
        #axis[1].legend(loc='upper right')
        axis[1].legend(bbox_to_anchor=(0.75, -0.5), ncol=2)
        axis[1].set_title("All - Found")
        plt.axvline(x = index, color = 'orange')
    
        stop_num -= 1
        if stop_num == 0:
            break
        plt.show()

        if saveName is not None:
            figure.savefig(f'{save_rep}/figures/get_weights/{saveName}_corrComp_{secret}.png')
            
        last_found_indx = time_sample

## Finish

In [12]:
print("✔️ The General functions/agregate succesfuly imported.")

✔️ The General functions/agregate succesfuly imported.
